<a href="https://colab.research.google.com/github/didtjr5080/kakao-voice-ai/blob/main/colab_train_all_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📱 카카오톡 + 인스타그램 통합 학습

## 순서
1. 환경 설정
2. 데이터 업로드 (CSV + TXT)
3. 데이터 전처리
4. 모델 학습
5. 다운로드

In [ ]:
# ===== 1. 환경 설정 =====
print('🔧 Installing packages...')
!pip install -q transformers datasets peft accelerate bitsandbytes sentencepiece

In [ ]:
# GPU 확인
import torch
print(f'GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Not available"}')
print(f'CUDA: {torch.cuda.is_available()}')

In [ ]:
# ===== 2. 데이터 파서 클래스 =====

import re
import pandas as pd
from typing import List, Tuple

class UnifiedChatParser:
    """카카오톡 + 인스타그램 통합 파서"""

    def __init__(self, your_name: str = "나"):
        self.your_name = your_name

    def parse_kakao_csv(self, csv_path: str) -> List[Tuple[str, str]]:
        """카카오톡 CSV 파싱"""
        print(f"\n📱 Parsing KakaoTalk: {csv_path}")

        df = pd.read_csv(csv_path)
        pairs = []

        prev_msg = None
        prev_sender = None

        for _, row in df.iterrows():
            sender = row['User']
            message = str(row['Message']).strip()

            if not message or message == 'nan' or len(message) < 1 or len(message) > 200:
                continue

            message = self._clean(message)

            if prev_msg and prev_sender != sender:
                pairs.append((prev_msg, message))

            prev_msg = message
            prev_sender = sender

        print(f"  ✅ {len(pairs)} pairs extracted")
        return pairs

    def parse_instagram_txt(self, txt_path: str) -> List[Tuple[str, str]]:
        """인스타그램 TXT 파싱"""
        print(f"\n📷 Parsing Instagram: {txt_path}")

        with open(txt_path, 'r', encoding='utf-8') as f:
            content = f.read()

        pairs = []
        messages = []

        # 패턴: [이름] 날짜\n\t메시지
        pattern = r'\[([^\]]+)\]\s+(\d{4}년[^\n]+)\n\s+(.+?)(?=\n\[|\Z)'

        for match in re.finditer(pattern, content, re.DOTALL):
            sender = match.group(1).strip()
            message = match.group(3).strip()

            # 탭, 줄바꿈 정리
            message = ' '.join(message.split())
            message = self._clean(message)

            if len(message) < 1 or len(message) > 200:
                continue

            messages.append({'sender': sender, 'text': message})

        # 대화 쌍 생성
        for i in range(len(messages) - 1):
            if messages[i]['sender'] != messages[i+1]['sender']:
                pairs.append((messages[i]['text'], messages[i+1]['text']))

        print(f"  ✅ {len(pairs)} pairs extracted")
        return pairs

    def _clean(self, text: str) -> str:
        """텍스트 정리"""
        # URL 제거
        text = re.sub(r'http[s]?://\S+', '', text)
        # 이메일 제거
        text = re.sub(r'\S+@\S+', '', text)
        # 이름 제거 (개인정보)
        text = re.sub(r'삼괴[가-힣]+', '', text)
        # 공백 정리
        text = ' '.join(text.split())
        return text.strip()

    def merge_and_deduplicate(self, all_pairs: List[List[Tuple[str, str]]]) -> List[Tuple[str, str]]:
        """병합 및 중복 제거"""
        print("\n🔄 Merging and deduplicating...")

        merged = []
        seen = set()

        for pairs in all_pairs:
            for inp, out in pairs:
                key = (inp.lower(), out.lower())
                if key not in seen:
                    seen.add(key)
                    merged.append((inp, out))

        print(f"  ✅ Total unique pairs: {len(merged)}")
        return merged

print('✅ Parser class loaded')

In [ ]:
# ===== 3. 데이터 업로드 =====

from google.colab import files
import os

os.makedirs('data', exist_ok=True)

print('📤 Upload ALL your chat files (CSV for KakaoTalk, TXT for Instagram):')
print('   You can select multiple files at once!')
uploaded = files.upload()

# 파일 분류
kakao_files = [f for f in uploaded.keys() if f.endswith('.csv')]
instagram_files = [f for f in uploaded.keys() if f.endswith('.txt')]

print(f"\n✅ Uploaded:")
print(f"   📱 KakaoTalk: {len(kakao_files)} files")
print(f"   📷 Instagram: {len(instagram_files)} files")

# 파일 이동
for f in uploaded.keys():
    !mv "{f}" data/

In [ ]:
# ===== 4. 데이터 전처리 =====

parser = UnifiedChatParser(your_name="나")  # 본인 이름으로 변경

all_pairs = []

# 카카오톡 파일 파싱
for csv_file in kakao_files:
    try:
        pairs = parser.parse_kakao_csv(f'data/{csv_file}')
        all_pairs.append(pairs)
    except Exception as e:
        print(f"⚠️ Error: {e}")

# 인스타그램 파일 파싱
for txt_file in instagram_files:
    try:
        pairs = parser.parse_instagram_txt(f'data/{txt_file}')
        all_pairs.append(pairs)
    except Exception as e:
        print(f"⚠️ Error: {e}")

# 병합 및 중복 제거
training_pairs = parser.merge_and_deduplicate(all_pairs)

# 샘플 출력
print("\n📝 Sample conversations:")
for i, (inp, out) in enumerate(training_pairs[:10], 1):
    print(f"{i:2}. User: {inp}")
    print(f"    Bot:  {out}\n")

In [ ]:
# ===== 5. 모델 클래스 정의 =====

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset

class LightweightChatbot:
    def __init__(self, model_name="skt/kogpt2-base-v2"):
        self.model_name = model_name
        self.tokenizer = None
        self.model = None

    def train(self, training_pairs, output_dir="./kakao_instagram_model", epochs=5, batch_size=4):
        print("\n" + "="*60)
        print("🚀 Starting Training")
        print("="*60)

        # 토크나이저
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        special_tokens = {'additional_special_tokens': ['<|user|>', '<|bot|>', '<|end|>']}
        self.tokenizer.add_special_tokens(special_tokens)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        # 모델 (4bit 양자화)
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4"
        )

        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            quantization_config=quantization_config,
            device_map="auto"
        )

        self.model.resize_token_embeddings(len(self.tokenizer))
        self.model = prepare_model_for_kbit_training(self.model)

        # LoRA 설정
        peft_config = LoraConfig(
            r=16,
            lora_alpha=32,
            target_modules=["c_attn", "c_proj"],
            lora_dropout=0.05,
            bias="none",
            task_type="CAUSAL_LM"
        )

        self.model = get_peft_model(self.model, peft_config)
        self.model.print_trainable_parameters()

        # 데이터셋 준비
        training_texts = [f"<|user|>{inp}<|bot|>{out}<|end|>" for inp, out in training_pairs]
        dataset = Dataset.from_dict({'text': training_texts})

        def tokenize(examples):
            return self.tokenizer(
                examples['text'],
                truncation=True,
                max_length=256,
                padding='max_length'
            )

        tokenized = dataset.map(tokenize, batched=True, remove_columns=['text'])

        # 학습 설정
        training_args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=epochs,
            per_device_train_batch_size=batch_size,
            gradient_accumulation_steps=4,
            learning_rate=2e-4,
            fp16=True,
            logging_steps=50,
            save_steps=500,
            save_total_limit=2,
            warmup_steps=100,
            gradient_checkpointing=True,
            optim="adamw_torch",
            report_to="none"
        )

        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=tokenized,
            data_collator=DataCollatorForLanguageModeling(self.tokenizer, mlm=False)
        )

        # 학습 시작
        print(f"\n📊 Training on {len(training_pairs)} conversation pairs")
        trainer.train()

        # 저장
        print(f"\n💾 Saving to {output_dir}")
        trainer.save_model(output_dir)
        self.tokenizer.save_pretrained(output_dir)

        print("\n✅ Training Complete!")
        return output_dir

print('✅ Model class loaded')

In [ ]:
# ===== 6. 학습 실행 =====

chatbot = LightweightChatbot()

output_dir = chatbot.train(
    training_pairs=training_pairs,
    output_dir="./kakao_instagram_model",
    epochs=5,           # 데이터 많으면 3-5로
    batch_size=4        # Colab GPU에 맞게 조정
)

In [ ]:
# ===== 7. 간단한 테스트 =====

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

print("🧪 Loading model for testing...")

tokenizer = AutoTokenizer.from_pretrained(output_dir)
base_model = AutoModelForCausalLM.from_pretrained("skt/kogpt2-base-v2")
base_model.resize_token_embeddings(len(tokenizer))
model = PeftModel.from_pretrained(base_model, output_dir)
model.eval()

def test_generate(user_input):
    prompt = f"<|user|>{user_input}<|bot|>"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=30,
            temperature=0.7,
            top_p=0.85,
            repetition_penalty=1.8,
            do_sample=True
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    if '<|bot|>' in response:
        response = response.split('<|bot|>')[-1].split('<|end|>')[0].strip()
    return response

# 테스트
test_inputs = ["안녕", "뭐해?", "ㅋㅋㅋ", "오늘 뭐 먹을까?", "게임하자"]

print("\n" + "="*60)
print("📝 Test Results")
print("="*60)

for inp in test_inputs:
    response = test_generate(inp)
    print(f"User: {inp}")
    print(f"Bot:  {response}")
    print("-" * 60)

In [ ]:
# ===== 8. 모델 다운로드 =====

print("📦 Zipping model...")
!zip -r kakao_instagram_model.zip {output_dir}

print("\n⬇️ Downloading...")
files.download('kakao_instagram_model.zip')

print("\n✅ Complete! Extract and use with your local app.py")

In [ ]:
# ===== 체크포인트 → 최종 모델 변환 (최종 수정) =====
from google.colab import drive, files
import torch
import os
import shutil

drive.mount("/content/drive")

CHECKPOINT = "/content/drive/MyDrive/kakao_final_training/kakao_final_model/checkpoint-3300"
OUTPUT_DIR = "/content/drive/MyDrive/kakao_final_training/kakao_final_model_step3300"

print("🔄 Converting checkpoint-3300 to standalone model...")

# 출력 폴더 생성
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 체크포인트 파일 확인
print("\n📂 Checkpoint files:")
!ls -lh {CHECKPOINT}

# ========================================
# 방법 1: 단순 복사 (가장 안전)
# ========================================
print("\n📋 Method 1: Direct copy...")

# 필요한 파일들만 복사
files_to_copy = [
    "adapter_config.json",
    "adapter_model.bin",
    "adapter_model.safetensors",
    "tokenizer_config.json",
    "tokenizer.json",
    "special_tokens_map.json",
    "vocab.json",
    "merges.txt",
    "added_tokens.json"
]

copied_files = []
for filename in files_to_copy:
    src = os.path.join(CHECKPOINT, filename)
    dst = os.path.join(OUTPUT_DIR, filename)
    if os.path.exists(src):
        shutil.copy2(src, dst)
        copied_files.append(filename)
        print(f"  ✅ Copied: {filename}")

if len(copied_files) > 0:
    print(f"\n✅ Method 1 succeeded! Copied {len(copied_files)} files")

    print("\n📂 Output files:")
    !ls -lh {OUTPUT_DIR}

else:
    print("\n⚠️ Method 1 failed, trying Method 2...")

    # ========================================
    # 방법 2: 전체 폴더 복사
    # ========================================
    print("\n📋 Method 2: Full directory copy...")

    if os.path.exists(OUTPUT_DIR):
        shutil.rmtree(OUTPUT_DIR)

    shutil.copytree(CHECKPOINT, OUTPUT_DIR)
    print("✅ Method 2 succeeded!")

    print("\n📂 Output files:")
    !ls -lh {OUTPUT_DIR}

# ========================================
# 방법 3: 모델 로드 후 재저장 (선택)
# ========================================
try:
    print("\n📋 Method 3: Load and re-save (verification)...")

    from transformers import AutoTokenizer, AutoModelForCausalLM
    from peft import PeftModel, PeftConfig

    # 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR)
    print("  ✅ Tokenizer loaded")

    # PEFT config 로드
    peft_config = PeftConfig.from_pretrained(OUTPUT_DIR)
    print("  ✅ PEFT config loaded")

    # 베이스 모델 로드
    base_model = AutoModelForCausalLM.from_pretrained(
        "skt/kogpt2-base-v2",
        torch_dtype=torch.float16,
        device_map="auto"
    )
    base_model.resize_token_embeddings(len(tokenizer))
    print("  ✅ Base model loaded")

    # PEFT 어댑터 로드 (로컬 경로로)
    model = PeftModel.from_pretrained(
        base_model,
        OUTPUT_DIR,
        is_trainable=False
    )
    print("  ✅ PEFT model loaded")

    # 재저장 (검증)
    VERIFIED_DIR = OUTPUT_DIR + "_verified"
    os.makedirs(VERIFIED_DIR, exist_ok=True)

    model.save_pretrained(VERIFIED_DIR)
    tokenizer.save_pretrained(VERIFIED_DIR)

    print(f"  ✅ Verified and saved to: {VERIFIED_DIR}")

    # 검증된 버전을 최종 출력으로 사용
    OUTPUT_DIR = VERIFIED_DIR

except Exception as e:
    print(f"\n⚠️ Method 3 failed (optional): {e}")
    print("Using Method 1/2 output (still usable)")

# ========================================
# ZIP 생성 및 다운로드
# ========================================
print("\n📦 Creating ZIP...")

# ZIP 파일명 결정
zip_name = os.path.basename(OUTPUT_DIR)
zip_path = f"/content/{zip_name}.zip"

!cd /content/drive/MyDrive/kakao_final_training && \
  zip -r {zip_path} {os.path.basename(OUTPUT_DIR)}/

print("\n📊 ZIP size:")
!ls -lh {zip_path}

print("\n📂 Final model structure:")
!ls -lh {OUTPUT_DIR}

print("\n⬇️ Downloading...")
files.download(zip_path)

print("\n" + "="*60)
print("✅ COMPLETE!")
print("="*60)
print(f"\nDownloaded: {zip_name}.zip")
print(f"\nExtract to: ./models/kakao_final_training/{zip_name}/")
print("\nUsage in tests/quick_test.py:")
print(f'  chatbot.load_model("./models/kakao_final_training/{zip_name}", use_lora=True)')

Mounted at /content/drive
🔄 Converting checkpoint-3300 to standalone model...

📂 Checkpoint files:
total 18M
-rw------- 1 root root  972 Jan 26 04:54 adapter_config.json
-rw------- 1 root root   90 Jan 26 04:54 added_tokens.json
-rw------- 1 root root 485K Jan 26 04:54 merges.txt
-rw------- 1 root root  13M Jan 26 04:54 optimizer.pt
-rw------- 1 root root 5.1K Jan 26 04:54 README.md
-rw------- 1 root root  15K Jan 26 04:54 rng_state.pth
-rw------- 1 root root 1.4K Jan 26 04:54 scaler.pt
-rw------- 1 root root 1.5K Jan 26 04:54 scheduler.pt
-rw------- 1 root root  592 Jan 26 04:54 special_tokens_map.json
-rw------- 1 root root  64K Jan 26 04:54 tokenizer_config.json
-rw------- 1 root root 3.4M Jan 26 04:54 tokenizer.json
-rw------- 1 root root  12K Jan 26 04:54 trainer_state.json
-rw------- 1 root root 5.8K Jan 26 04:54 training_args.bin
-rw------- 1 root root 870K Jan 26 04:54 vocab.json

📋 Method 1: Direct copy...
  ✅ Copied: adapter_config.json
  ✅ Copied: tokenizer_config.json
  ✅ C

config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


...

model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

-rw-r--r-- 1 root root 1.2M Jan 26 23:31 /content/kakao_final_model_step3300.zip

📂 Final model structure:
total 4.7M
-rw------- 1 root root  972 Jan 26 04:54 adapter_config.json
-rw------- 1 root root   90 Jan 26 04:54 added_tokens.json
-rw------- 1 root root 485K Jan 26 04:54 merges.txt
-rw------- 1 root root  592 Jan 26 04:54 special_tokens_map.json
-rw------- 1 root root  64K Jan 26 04:54 tokenizer_config.json
-rw------- 1 root root 3.4M Jan 26 04:54 tokenizer.json
-rw------- 1 root root 870K Jan 26 04:54 vocab.json

⬇️ Downloading...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ COMPLETE!

Downloaded: kakao_final_model_step3300.zip

Extract to: ./models/kakao_final_training/kakao_final_model_step3300/

Usage in tests/quick_test.py:
  chatbot.load_model("./models/kakao_final_training/kakao_final_model_step3300", use_lora=True)


In [ ]:
# ===== Step 1: 패키지만 설치 후 런타임 재시작 =====
!pip install -q -U bitsandbytes accelerate transformers peft datasets sentencepiece

# 런타임 재시작
import os
os.kill(os.getpid(), 9)

In [ ]:
# ============================================================
# 카카오톡 챗봇 완전 재학습 (런타임 재시작 후)
# ============================================================

from google.colab import drive
import os, time, torch
from IPython.display import display, Javascript

# Drive 마운트
drive.mount("/content/drive")
print("✅ Drive mounted")

# 버전 확인
import bitsandbytes as bnb
import transformers
print(f"🔧 bitsandbytes: {bnb.__version__}")
print(f"🔧 transformers: {transformers.__version__}")

# GPU 확인
print(f"🎮 GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU'}")

# 세션 유지
display(Javascript('''
    function ClickConnect(){
        console.log("⏰ " + new Date().toLocaleTimeString());
    }
    setInterval(ClickConnect, 60000)
'''))

# 프로젝트 설정
PROJECT_DIR = "/content/drive/MyDrive/kakao_retrain_epoch7"
MODEL_DIR = os.path.join(PROJECT_DIR, "final_model")
LOG_FILE = os.path.join(PROJECT_DIR, "training.log")

os.makedirs(PROJECT_DIR, exist_ok=True)

def log(msg):
    timestamp = time.strftime("%H:%M:%S")
    log_msg = f"[{timestamp}] {msg}"
    print(log_msg)
    try:
        with open(LOG_FILE, "a", encoding="utf-8") as f:
            f.write(log_msg + "\n")
    except:
        pass

log("="*60)
log("🚀 Full Retraining (Epoch 7)")
log("="*60)

# 데이터 로드
import json
DATA_PATH = "/content/drive/MyDrive/kakao_final_training/training_pairs_final.json"

with open(DATA_PATH, "r") as f:
    data = json.load(f)

if isinstance(data[0], dict):
    if 'user' in data[0]:
        pairs = [(item['user'], item['bot']) for item in data]
    else:
        pairs = [(item['u'], item['b']) for item in data]
else:
    pairs = data

log(f"✅ Loaded {len(pairs)} pairs")

# 백업
with open(os.path.join(PROJECT_DIR, "data.json"), "w") as f:
    json.dump([{"user": u, "bot": b} for u, b in pairs], f, ensure_ascii=False)

# 모델 학습
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,
    TrainingArguments, Trainer, DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset

MODEL_NAME = "skt/kogpt2-base-v2"

log("\n📖 Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.add_special_tokens({'additional_special_tokens': ['<|user|>', '<|bot|>', '<|end|>']})
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
log(f"   Vocab: {len(tokenizer)}")

log("\n🤖 Loading model...")
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quant_config,
    device_map="auto"
)
model.resize_token_embeddings(len(tokenizer))
model = prepare_model_for_kbit_training(model)
log("   ✅ Model loaded")

log("\n🔧 Applying LoRA...")
peft_config = LoraConfig(
    r=16, lora_alpha=32,
    target_modules=["c_attn", "c_proj"],
    lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

log("\n📊 Preparing dataset...")
texts = [f"<|user|>{u}<|bot|>{b}<|end|>" for u, b in pairs]
dataset = Dataset.from_dict({'text': texts})

def tokenize(ex):
    return tokenizer(ex['text'], truncation=True, max_length=256, padding='max_length')

tokenized = dataset.map(tokenize, batched=True, remove_columns=['text'])
log(f"   ✅ {len(tokenized)} samples ready")

# 학습 설정
batch_size = 2
grad_accum = 8

args = TrainingArguments(
    output_dir=MODEL_DIR,
    num_train_epochs=7,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=grad_accum,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=50,
    save_steps=1000,
    save_total_limit=3,
    warmup_steps=100,
    gradient_checkpointing=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

total_steps = (len(pairs) * 7) // (batch_size * grad_accum)
log(f"\n⏱️ Estimated: {total_steps//200}~{total_steps//100} min ({total_steps//100//60+1}h)")
log(f"   Total steps: ~{total_steps}")
log("\n" + "="*60)
log("🚀 TRAINING STARTED")
log("💤 Check back in 4 hours")
log("="*60)

start_time = time.time()
trainer.train()

elapsed = (time.time() - start_time) / 60
log(f"\n✅ COMPLETE! ({elapsed:.1f} min)")

# 저장
trainer.save_model(MODEL_DIR)
tokenizer.save_pretrained(MODEL_DIR)
log(f"💾 Saved to: {MODEL_DIR}")

# 테스트
log("\n🧪 Testing...")
base = AutoModelForCausalLM.from_pretrained("skt/kogpt2-base-v2", torch_dtype=torch.float16, device_map="auto")
base.resize_token_embeddings(len(tokenizer))

from peft import PeftModel
model = PeftModel.from_pretrained(base, MODEL_DIR)
model.eval()

def gen(text):
    prompt = f"<|user|>{text}<|bot|>"
    inp = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(**inp, max_new_tokens=30, temperature=0.7, top_p=0.85, repetition_penalty=1.8, do_sample=True)
    resp = tokenizer.decode(out[0], skip_special_tokens=False)
    if '<|bot|>' in resp:
        resp = resp.split('<|bot|>')[-1].split('<|end|>')[0].strip()
    return resp

for t in ["안녕", "뭐해?", "ㅋㅋㅋ", "심심해"]:
    log(f"U: {t}")
    log(f"B: {gen(t)}\n")

# 다운로드
from google.colab import files

log("\n📦 Preparing download...")
!cd {MODEL_DIR} && rm -rf checkpoint-*
!cd {PROJECT_DIR} && zip -r /content/final.zip final_model/

log("⬇️ Downloading...")
files.download('/content/final.zip')

log("\n✅ ALL DONE!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Drive mounted
🔧 bitsandbytes: 0.49.1
🔧 transformers: 5.0.0
🎮 GPU: Tesla T4


<IPython.core.display.Javascript object>

[23:39:29] ============================================================
[23:39:29] 🚀 Full Retraining (Epoch 7)
[23:39:29] ============================================================
[23:39:29] ✅ Loaded 18150 pairs
[23:39:35] 
📖 Loading tokenizer...
[23:39:39]    Vocab: 51204
[23:39:39] 
🤖 Loading model...


Loading weights:   0%|          | 0/149 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie transformer.wte.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
GPT2LMHeadModel LOAD REPORT from: skt/kogpt2-base-v2
Key                                     | Status     |  | 
----------------------------------------+------------+--+-
transformer.h.{0...11}.attn.bias        | UNEXPECTED |  | 
transformer.h.{0...11}.attn.masked_bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal dis

...

The tied weights mapping and config for this model specifies to tie transformer.wte.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
GPT2LMHeadModel LOAD REPORT from: skt/kogpt2-base-v2
Key                                     | Status     |  | 
----------------------------------------+------------+--+-
transformer.h.{0...11}.attn.bias        | UNEXPECTED |  | 
transformer.h.{0...11}.attn.masked_bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2285: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


[02:14:50] U: 안녕
[02:14:51] B: ��:#�.pdf?!�5,000(2-3)1/72523800

[02:14:51] U: 뭐해?
[02:14:52] B: ��3,4ㅎ��510giettyummyr�6248270

[02:14:52] U: ㅋㅋㅋ
[02:14:52] B: �����??
�...!���.#6-17418515225330

[02:14:52] U: 심심해
[02:14:53] B: ��3,4�1-5��2(9)10800~13506.25%�

[02:14:53] 
📦 Preparing download...
  adding: final_model/ (stored 0%)
  adding: final_model/README.md (deflated 65%)
  adding: final_model/adapter_model.safetensors (deflated 7%)
  adding: final_model/adapter_config.json (deflated 57%)
  adding: final_model/tokenizer_config.json (deflated 50%)
  adding: final_model/tokenizer.json (deflated 81%)
  adding: final_model/training_args.bin (deflated 53%)
[02:15:11] ⬇️ Downloading...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[02:15:11] 
✅ ALL DONE!


In [2]:
# ============================================================
# KoCulture-Dialogues QLoRA 재학습 + 테스트 (프롬프트 없이)
# - 데이터: huggingface-KREW/KoCulture-Dialogues
# - 베이스: skt/kogpt2-base-v2
# - 포맷: <|user|>...<|bot|>...<|end|>
# ============================================================
# ============================================================
# 필수 라이브러리 설치 (Colab GPU 런타임용)
# ============================================================
!pip -q install -U bitsandbytes
!pip -q install -U transformers datasets peft trl accelerate sentencepiece

from google.colab import drive
import os, time, torch
from IPython.display import display, Javascript

# -----------------------------
# 0) Drive mount & 환경 확인
# -----------------------------
drive.mount("/content/drive")
print("✅ Drive mounted")

import bitsandbytes as bnb
import transformers
print(f"🔧 bitsandbytes: {bnb.__version__}")
print(f"🔧 transformers: {transformers.__version__}")
print(f"🎮 GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU'}")

# 세션 유지(선택)
display(Javascript('''
    function ClickConnect(){
        console.log("⏰ " + new Date().toLocaleTimeString());
    }
    setInterval(ClickConnect, 60000)
'''))

# -----------------------------
# 1) 프로젝트 설정
# -----------------------------
PROJECT_DIR = "/content/drive/MyDrive/koculture_retrain"
MODEL_DIR = os.path.join(PROJECT_DIR, "final_model")
LOG_FILE = os.path.join(PROJECT_DIR, "training.log")

os.makedirs(PROJECT_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

def log(msg):
    timestamp = time.strftime("%H:%M:%S")
    log_msg = f"[{timestamp}] {msg}"
    print(log_msg)
    try:
        with open(LOG_FILE, "a", encoding="utf-8") as f:
            f.write(log_msg + "\n")
    except:
        pass

log("="*60)
log("🚀 KoCulture-Dialogues QLoRA Retraining (NO PROMPT)")
log("="*60)

# -----------------------------
# 2) 라이브러리 설치(필요 시)
# -----------------------------
# ⚠️ Colab에서 최초 1회만 실행되도록 주석 해제해서 사용
# !pip -q install -U datasets peft trl accelerate bitsandbytes transformers sentencepiece

# -----------------------------
# 3) 데이터 로드 (HuggingFace Datasets)
# -----------------------------
from datasets import load_dataset, Dataset



DATASET_NAME = "huggingface-KREW/KoCulture-Dialogues"
log(f"\n📦 Loading dataset: {DATASET_NAME}")
ds = load_dataset(DATASET_NAME, verification_mode="no_checks")


# KoCulture-Dialogues는 보통 train만 있는 경우가 많아서 split 생성
if "validation" not in ds:
    split = ds["train"].train_test_split(test_size=0.1, seed=42)
    train_raw = split["train"]
    val_raw = split["test"]
else:
    train_raw = ds["train"]
    val_raw = ds["validation"]

log(f"✅ train: {len(train_raw)}, val: {len(val_raw)}")

# (선택) title을 user 쪽에 붙일지 여부
# 프롬프트는 아니고 "원문 데이터 필드 활용" 수준이라 원하는 경우만 True
USE_TITLE_HINT = False

def pack_pair(ex):
    title = (ex.get("title", "") or "").strip()
    q = (ex.get("question", "") or "").strip()
    a = (ex.get("answer", "") or "").strip()

    # 질문/답변이 비어있으면 제거
    if not q or not a:
        return None

    if USE_TITLE_HINT and title:
        user = f"[주제] {title}\n{q}"
    else:
        user = q

    return (user, a)

train_pairs = []
for ex in train_raw:
    p = pack_pair(ex)
    if p is not None:
        train_pairs.append(p)

val_pairs = []
for ex in val_raw:
    p = pack_pair(ex)
    if p is not None:
        val_pairs.append(p)

log(f"✅ Packed pairs -> train: {len(train_pairs)}, val: {len(val_pairs)}")

# 백업 저장(Drive)
import json
with open(os.path.join(PROJECT_DIR, "train_pairs.json"), "w", encoding="utf-8") as f:
    json.dump([{"user": u, "bot": b} for u, b in train_pairs], f, ensure_ascii=False, indent=2)
with open(os.path.join(PROJECT_DIR, "val_pairs.json"), "w", encoding="utf-8") as f:
    json.dump([{"user": u, "bot": b} for u, b in val_pairs], f, ensure_ascii=False, indent=2)

# -----------------------------
# 4) 토크나이저/모델 로드 (4bit QLoRA)
# -----------------------------
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,
    TrainingArguments, Trainer, DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

MODEL_NAME = "skt/kogpt2-base-v2"

log("\n📖 Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 특수 토큰 추가 (프롬프트 없이 대화 구분용)
SPECIAL_TOKENS = ['<|user|>', '<|bot|>', '<|end|>']
tokenizer.add_special_tokens({'additional_special_tokens': SPECIAL_TOKENS})

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

log(f"   Vocab size: {len(tokenizer)}")
log(f"   pad_token: {tokenizer.pad_token} / eos_token: {tokenizer.eos_token}")

log("\n🤖 Loading model (4bit)...")
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quant_config,
    device_map="auto"
)
model.resize_token_embeddings(len(tokenizer))

# k-bit 학습 준비
model = prepare_model_for_kbit_training(model)

log("   ✅ Model loaded & prepared")

# -----------------------------
# 5) LoRA 적용
# -----------------------------
log("\n🔧 Applying LoRA...")

# GPT2 계열(KoGPT2)은 보통 c_attn, c_proj가 핵심
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["c_attn", "c_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# -----------------------------
# 6) 학습 데이터 구성 (프롬프트 없이)
# -----------------------------
log("\n📊 Preparing dataset...")

# 완전 프롬프트 없이: <|user|>...<|bot|>...<|end|>
def build_text(u, b):
    return f"<|user|>{u}<|bot|>{b}<|end|>"

train_texts = [build_text(u, b) for u, b in train_pairs]
val_texts   = [build_text(u, b) for u, b in val_pairs]

train_ds = Dataset.from_dict({"text": train_texts})
val_ds   = Dataset.from_dict({"text": val_texts})

MAX_LEN = 256  # 1650급 VRAM 고려: 256~384 권장

def tokenize_fn(ex):
    return tokenizer(
        ex["text"],
        truncation=True,
        max_length=MAX_LEN,
        padding="max_length",
    )

train_tok = train_ds.map(tokenize_fn, batched=True, remove_columns=["text"])
val_tok   = val_ds.map(tokenize_fn, batched=True, remove_columns=["text"])

log(f"   ✅ train samples: {len(train_tok)}, val samples: {len(val_tok)}")
log(f"   ✅ max_length: {MAX_LEN}")

# -----------------------------
# 7) 학습 설정 & 실행
# -----------------------------
EPOCHS = 3
BATCH_SIZE = 2
GRAD_ACCUM = 8
LR = 2e-4

args = TrainingArguments(
    output_dir=MODEL_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,
    learning_rate=LR,
    fp16=True,
    logging_steps=50,
    save_steps=800,
    save_total_limit=2,
    warmup_steps=100,
    gradient_checkpointing=True,
    evaluation_strategy="steps",
    eval_steps=800,
    report_to="none",
    dataloader_pin_memory=False,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

total_steps = (len(train_pairs) * EPOCHS) // max(1, (BATCH_SIZE * GRAD_ACCUM))
log("\n" + "="*60)
log("🚀 TRAINING STARTED")
log(f"   Base model: {MODEL_NAME}")
log(f"   Epochs: {EPOCHS}, batch: {BATCH_SIZE}, grad_accum: {GRAD_ACCUM}, lr: {LR}")
log(f"   Total steps (rough): ~{total_steps}")
log("="*60)

start_time = time.time()
trainer.train()
elapsed = (time.time() - start_time) / 60

log(f"\n✅ COMPLETE! ({elapsed:.1f} min)")

# 저장
trainer.save_model(MODEL_DIR)
tokenizer.save_pretrained(MODEL_DIR)
log(f"💾 Saved to: {MODEL_DIR}")

# -----------------------------
# 8) 테스트 (LoRA 로드 후 생성)
# -----------------------------
log("\n🧪 Testing...")

base = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)
base.resize_token_embeddings(len(tokenizer))

from peft import PeftModel
test_model = PeftModel.from_pretrained(base, MODEL_DIR)
test_model.eval()

def gen(text, max_new_tokens=60, temperature=0.7, top_p=0.85, rep_pen=1.6):
    # 프롬프트 없음: user/bot 토큰만 사용
    prompt = f"<|user|>{text}<|bot|>"
    inp = tokenizer(prompt, return_tensors="pt").to(test_model.device)

    with torch.no_grad():
        out = test_model.generate(
            **inp,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            repetition_penalty=rep_pen,
            do_sample=True,
            eos_token_id=tokenizer.convert_tokens_to_ids("<|end|>"),
            pad_token_id=tokenizer.pad_token_id,
        )

    resp = tokenizer.decode(out[0], skip_special_tokens=False)

    # <|bot|> 이후 ~ <|end|> 전까지만 추출
    if "<|bot|>" in resp:
        resp = resp.split("<|bot|>", 1)[-1]
    if "<|end|>" in resp:
        resp = resp.split("<|end|>", 1)[0]
    return resp.strip()

tests = [
    "안녕",
    "킹받네가 무슨 뜻이야?",
    "요즘 밈으로 '중꺾마'는 어떤 뉘앙스야?",
    "ㅋㅋㅋ",
    "심심해",
]

for t in tests:
    log(f"U: {t}")
    log(f"B: {gen(t)}\n")

# -----------------------------
# 9) zip 다운로드 준비
# -----------------------------
from google.colab import files

log("\n📦 Preparing download...")
!cd {MODEL_DIR} && rm -rf checkpoint-* || true
!cd {PROJECT_DIR} && zip -r /content/koculture_final.zip final_model/ >/dev/null

log("⬇️ Downloading...")
files.download("/content/koculture_final.zip")

log("\n✅ ALL DONE!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Drive mounted
🔧 bitsandbytes: 0.49.1
🔧 transformers: 5.0.0
🎮 GPU: Tesla T4


<IPython.core.display.Javascript object>

[09:30:37] ============================================================
[09:30:37] 🚀 KoCulture-Dialogues QLoRA Retraining (NO PROMPT)
[09:30:37] ============================================================
[09:30:37] 
📦 Loading dataset: huggingface-KREW/KoCulture-Dialogues


Generating train split:   0%|          | 0/3456 [00:00<?, ? examples/s]

[09:30:39] ✅ train: 9320, val: 1036
[09:30:39] ✅ Packed pairs -> train: 9320, val: 1036
[09:30:42] 
📖 Loading tokenizer...


...

[09:30:54]    ✅ Model loaded & prepared
[09:30:54] 
🔧 Applying LoRA...
trainable params: 1,622,016 || all params: 126,789,120 || trainable%: 1.2793
[09:30:54] 
📊 Preparing dataset...


Map:   0%|          | 0/9320 [00:00<?, ? examples/s]

Map:   0%|          | 0/1036 [00:00<?, ? examples/s]

[09:31:01]    ✅ train samples: 9320, val samples: 1036
[09:31:01]    ✅ max_length: 256


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'